# Importações

In [ ]:
import bs4 as bs
import urllib.request
import re
import nltk
import numpy as np
import random
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import spacy
spacy.__version__

'2.2.4'

In [ ]:
nltk.__version__

'3.2.5'

In [ ]:
!python3 -m spacy download pt

     |████████████████████████████████| 21.2 MB 1.3 MB/s 
  Created wheel for pt-core-news-sm: filename=pt_core_news_sm-2.2.5-py3-none-any.whl size=21186281 sha256=a4dc1c098d01b170976345094bf2abc2b66c8fd28f08033d9d106ea1a39aef57
  Stored in directory: /tmp/pip-ephem-wheel-cache-oe7cbbzn/wheels/c3/f9/0c/5c014a36941a00f5df5fc0756cb961d7c457a978e697a6ce3b
Successfully built pt-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('pt_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/pt_core_news_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/pt
You can now load the model via spacy.load('pt')


# Carregamento e pré-processamento

In [ ]:
dados = urllib.request.urlopen('https://pt.wikipedia.org/wiki/Intelig%C3%AAncia_artificial')

In [ ]:
dados = dados.read()

In [ ]:
dados_html = bs.BeautifulSoup(dados, 'lxml')

In [ ]:
paragrafos = dados_html.find_all('p')

In [ ]:
len(paragrafos)

105

In [ ]:
conteudo = ''
for p in paragrafos:
  conteudo += p.text

In [ ]:
conteudo

'\n\t\tPáginas para editores conectados saiba mais\nInteligência artificial (por vezes mencionada pela sigla em português IA ou pela sigla em inglês AI - artificial intelligence) é a inteligência similar à humana exibida por sistemas de software, além de também ser um campo de estudo acadêmico. Os principais pesquisadores e livros didáticos definem o campo como "o estudo e projeto de agentes inteligentes", onde um agente inteligente é um sistema que percebe seu ambiente e toma atitudes que maximizam suas chances de sucesso. Andreas Kaplan e Michael Haenlein definem a inteligência artificial como “uma capacidade do sistema para interpretar corretamente dados externos, aprender a partir desses dados e utilizar essas aprendizagens para atingir objetivos e tarefas específicas através de adaptação flexível”.[1] John McCarthy, quem cunhou o termo em 1956 ("numa conferência de especialistas celebrada em Darmouth Colege" Gubern, Román: O Eros Eletrónico), a define como "a ciência e engenharia 

In [ ]:
conteudo = conteudo.lower()

In [ ]:
lista_sentencas = nltk.sent_tokenize(conteudo)

In [ ]:
lista_sentencas

['\n\t\tpáginas para editores conectados saiba mais\ninteligência artificial (por vezes mencionada pela sigla em português ia ou pela sigla em inglês ai - artificial intelligence) é a inteligência similar à humana exibida por sistemas de software, além de também ser um campo de estudo acadêmico.',
 'os principais pesquisadores e livros didáticos definem o campo como "o estudo e projeto de agentes inteligentes", onde um agente inteligente é um sistema que percebe seu ambiente e toma atitudes que maximizam suas chances de sucesso.',
 'andreas kaplan e michael haenlein definem a inteligência artificial como “uma capacidade do sistema para interpretar corretamente dados externos, aprender a partir desses dados e utilizar essas aprendizagens para atingir objetivos e tarefas específicas através de adaptação flexível”.',
 '[1] john mccarthy, quem cunhou o termo em 1956 ("numa conferência de especialistas celebrada em darmouth colege" gubern, román: o eros eletrónico), a define como "a ciência

In [ ]:
len(lista_sentencas)

333

In [ ]:
pln = spacy.load('pt')
pln

In [ ]:
stop = spacy.lang.pt.stop_words.STOP_WORDS

In [ ]:
type(stop)

set

In [ ]:
len(stop)

413

In [ ]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
def preprocessamento(texto):
  texto = re.sub(r"https?://[A-Za-z0-9./]+",' ',texto)
  texto = re.sub(r" +", ' ', texto)
  documento = pln(texto)
  lista = []
  for token in documento:
    lista.append(token.lemma_)
  lista = [palavra for palavra in lista if palavra not in stop and palavra not in string.punctuation]
  l = ' '.join([str(elemento) for elemento in lista if not elemento.isdigit()])
  return l

In [ ]:
teste = 'https://www.youtube.com ' + lista_sentencas[1]
teste

'https://www.youtube.com os principais pesquisadores e livros didáticos definem o campo como "o estudo e projeto de agentes inteligentes", onde um agente inteligente é um sistema que percebe seu ambiente e toma atitudes que maximizam suas chances de sucesso.'

In [ ]:
resultado = preprocessamento(teste)
resultado

'  o principal pesquisador e livro didáticos definir o campar comer o estudar e projeto agente inteligente agente inteligente perceber ambientar e tomar atitude maximizar suar chance sucesso'

In [ ]:
type(resultado)

str

In [ ]:
lista_sentencas_preprocessadas = []
for i in range(len(lista_sentencas)):
  lista_sentencas_preprocessadas.append(lista_sentencas[i])

In [ ]:
for _ in range(5):
  i = random.randint(0, len(lista_sentencas) - 1)
  print(lista_sentencas[i])
  print(lista_sentencas_preprocessadas[i])
  print('-----')

o watson tem se esforçado para obter sucesso e adoção na área de saúde.
o watson tem se esforçado para obter sucesso e adoção na área de saúde.
-----
[31][32]
enquanto que o progresso direcionado ao objetivo final de uma inteligência similar à humana tem sido lento,  muitas derivações surgiram no processo.
[31][32]
enquanto que o progresso direcionado ao objetivo final de uma inteligência similar à humana tem sido lento,  muitas derivações surgiram no processo.
-----
[101] para trabalhadores de manuseio manual de materiais, análises preditivas e inteligência artificial podem ser usadas para reduzir lesões musculoesqueléticas.
[101] para trabalhadores de manuseio manual de materiais, análises preditivas e inteligência artificial podem ser usadas para reduzir lesões musculoesqueléticas.
-----
ou seja, até que os fluxos de palavras produzidas pelo computador e pelo sujeito humano sejam praticamente indistinguíveis para um examinador humano.
ou seja, até que os fluxos de palavras produzida

In [ ]:
texto_entrada = ('oi', 'olá', 'tudo bem', 'eae', 'como vai')
texto_resposta = ('oi', 'olá', 'tudo bem', 'eae', 'como vai', 'seja bem vindo', 'legal te receber')

In [ ]:
def responder_saudacao(texto):
  for palavra in texto.split():
    if palavra.lower() in texto_entrada:
      return random.choice(texto_resposta)

In [ ]:
print(responder_saudacao('olá'))

legal te receber


# TF-IDF

In [ ]:
frases_test = lista_sentencas_preprocessadas[1:4]
frases_test

['o principal pesquisador e livro didáticos definir o campar comer o estudar e projeto agente inteligente agente inteligente perceber ambientar e tomar atitude maximizar suar chance sucesso',
 'andreas kaplan e michael haenlein definir o inteligência artificial comer “ umar capacidade parir interpretar corretamente externo aprender o d e utilizar aprendizagem parir atingir objetivos e tarefar específico adaptação flexível ”',
 'john mccarthy cunhar o conferência especialista celebrar darmouth colege gubern román o eros eletrónico o definir comer o ciência e engenhar produzir inteligente']

In [ ]:
frases_test.append(frases_test[0])
frases_test

['o principal pesquisador e livro didáticos definir o campar comer o estudar e projeto agente inteligente agente inteligente perceber ambientar e tomar atitude maximizar suar chance sucesso',
 'andreas kaplan e michael haenlein definir o inteligência artificial comer “ umar capacidade parir interpretar corretamente externo aprender o d e utilizar aprendizagem parir atingir objetivos e tarefar específico adaptação flexível ”',
 'john mccarthy cunhar o conferência especialista celebrar darmouth colege gubern román o eros eletrónico o definir comer o ciência e engenhar produzir inteligente',
 'o principal pesquisador e livro didáticos definir o campar comer o estudar e projeto agente inteligente agente inteligente perceber ambientar e tomar atitude maximizar suar chance sucesso']

In [ ]:
vetores_palavras = TfidfVectorizer()
palavras_vetorizadas = vetores_palavras.fit_transform(frases_test)

In [ ]:
print(vetores_palavras.get_feature_names())

['adaptação', 'agente', 'ambientar', 'andreas', 'aprender', 'aprendizagem', 'artificial', 'atingir', 'atitude', 'campar', 'capacidade', 'celebrar', 'chance', 'ciência', 'colege', 'comer', 'conferência', 'corretamente', 'cunhar', 'darmouth', 'definir', 'didáticos', 'eletrónico', 'engenhar', 'eros', 'especialista', 'específico', 'estudar', 'externo', 'flexível', 'gubern', 'haenlein', 'inteligente', 'inteligência', 'interpretar', 'john', 'kaplan', 'livro', 'maximizar', 'mccarthy', 'michael', 'objetivos', 'parir', 'perceber', 'pesquisador', 'principal', 'produzir', 'projeto', 'román', 'suar', 'sucesso', 'tarefar', 'tomar', 'umar', 'utilizar']


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
len(vetores_palavras.get_feature_names())

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


55

In [ ]:
print(vetores_palavras.vocabulary_)

{'principal': 45, 'pesquisador': 44, 'livro': 37, 'didáticos': 21, 'definir': 20, 'campar': 9, 'comer': 15, 'estudar': 27, 'projeto': 47, 'agente': 1, 'inteligente': 32, 'perceber': 43, 'ambientar': 2, 'tomar': 52, 'atitude': 8, 'maximizar': 38, 'suar': 49, 'chance': 12, 'sucesso': 50, 'andreas': 3, 'kaplan': 36, 'michael': 40, 'haenlein': 31, 'inteligência': 33, 'artificial': 6, 'umar': 53, 'capacidade': 10, 'parir': 42, 'interpretar': 34, 'corretamente': 17, 'externo': 28, 'aprender': 4, 'utilizar': 54, 'aprendizagem': 5, 'atingir': 7, 'objetivos': 41, 'tarefar': 51, 'específico': 26, 'adaptação': 0, 'flexível': 29, 'john': 35, 'mccarthy': 39, 'cunhar': 18, 'conferência': 16, 'especialista': 25, 'celebrar': 11, 'darmouth': 19, 'colege': 14, 'gubern': 30, 'román': 48, 'eros': 24, 'eletrónico': 22, 'ciência': 13, 'engenhar': 23, 'produzir': 46}


In [ ]:
print(vetores_palavras.idf_)

[1.91629073 1.51082562 1.51082562 1.91629073 1.91629073 1.91629073
 1.91629073 1.91629073 1.51082562 1.51082562 1.91629073 1.91629073
 1.51082562 1.91629073 1.91629073 1.         1.91629073 1.91629073
 1.91629073 1.91629073 1.         1.51082562 1.91629073 1.91629073
 1.91629073 1.91629073 1.91629073 1.51082562 1.91629073 1.91629073
 1.91629073 1.91629073 1.22314355 1.91629073 1.91629073 1.91629073
 1.91629073 1.51082562 1.51082562 1.91629073 1.91629073 1.91629073
 1.91629073 1.51082562 1.51082562 1.51082562 1.91629073 1.51082562
 1.91629073 1.51082562 1.51082562 1.91629073 1.51082562 1.91629073
 1.91629073]


In [ ]:
palavras_vetorizadas.todense()

matrix([[0.        , 0.42165656, 0.21082828, 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.21082828, 0.21082828,
         0.        , 0.        , 0.21082828, 0.        , 0.        ,
         0.13954508, 0.        , 0.        , 0.        , 0.        ,
         0.13954508, 0.21082828, 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.21082828, 0.        , 0.        ,
         0.        , 0.        , 0.34136732, 0.        , 0.        ,
         0.        , 0.        , 0.21082828, 0.21082828, 0.        ,
         0.        , 0.        , 0.        , 0.21082828, 0.21082828,
         0.21082828, 0.        , 0.21082828, 0.        , 0.21082828,
         0.21082828, 0.        , 0.21082828, 0.        , 0.        ],
        [0.20184672, 0.        , 0.        , 0.20184672, 0.20184672,
         0.20184672, 0.20184672, 0.20184672, 0.        , 0.        ,
         0.20184672, 0.        , 0.        , 0.        , 0.        ,
         0.10533199, 0.        , 

In [ ]:
palavras_vetorizadas.todense().shape

(4, 55)

# Similiridade do cosseno

In [ ]:
similaridade = cosine_similarity(palavras_vetorizadas[0], palavras_vetorizadas)
similaridade

array([[1.        , 0.02939712, 0.09101929, 1.        ]])

In [ ]:
similaridade.argsort()

array([[1, 2, 0, 3]])

In [ ]:
i = similaridade.argsort()[0][-2]
i

0

In [ ]:
i = i.flatten()
i

array([0])

# Função de resposta

In [ ]:
def responder(texto_usuario):
  resposta_chatbot = ''
  lista_sentencas_preprocessadas.append(texto_usuario)
  tfidf = TfidfVectorizer()
  palavras_vetorizadas = tfidf.fit_transform(lista_sentencas_preprocessadas)
  similaridade = cosine_similarity(palavras_vetorizadas[-1], palavras_vetorizadas)
  indice = similaridade.argsort()[0][-2]
  vetor_similar = similaridade.flatten()
  vetor_similar.sort()
  vetor_encontrado = vetor_similar[-2]

  if vetor_encontrado == 0:
    resposta_chatbot = resposta_chatbot + 'Desculpe, não entendi!'
    return resposta_chatbot
  else:
    resposta_chatbot = resposta_chatbot + lista_sentencas[indice]
    return resposta_chatbot

# Conversação

In [ ]:
continuar = True
print('Olá sou um chatbot e vou responder perguntas sobre inteligencia artificial, vamos começar me pergunte algo: ')
while continuar == True:
  texto_usuario = input()
  texto_usuario = texto_usuario.lower()
  if texto_usuario != 'sair':
    if responder_saudacao(texto_usuario) != None:
      print('ChatBot: ' + responder_saudacao(texto_usuario))
    else:
      print('ChatBot: ')
      print(responder(preprocessamento(texto_usuario)))
      lista_sentencas_preprocessadas.remove(preprocessamento(texto_usuario))
  else:
    continuar = False
    print('ChatBot: Até Breve!')


Olá sou um chatbot e vou responder perguntas sobre inteligencia artificial, vamos começar me pergunte algo: 
oi
ChatBot: como vai
alan turing
ChatBot: 
uma contribuição prática de alan turing foi o que se chamou depois de teste de turing (tt),[22] de 1950: em lugar de responder à pergunta "podem-se ter computadores inteligentes?"
inteligencia artificial
ChatBot: 
outra definição de inteligência artificial é a inteligência que surge de um "dispositivo artificial".
sair
ChatBot: Até Breve!
